# ch 5 오차전역전파법

수치미분은 단순하고 구현하기 쉽지만 계산 시간이 오래 걸린다는 단점이 있다.  
-> '오차역전파법'으로 가중치 매개변수의 기울기를 효율적으로 계산

수식을 통하여/계산 그래프

## 5.1 계산 그래프
계산 그래프: 계산 과정을 그래프로 나타낸 것. 노드, 에지로 표현. 국조적 계산을 전파함으로써 최종 결과를 얻는다.

계산 그래프의 이점: 국조적 계산(문제를 단순화)

## 5.2 단순계층구현

### 5.2.1 곱셈 계층

In [1]:
class MulLayer:
    
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y                
        out = x * y #곱해서 반환

        return out

    def backward(self, dout): #각 변수에 대한 미분값 구할 수 있음
        dx = dout * self.y  # x와 y를 바꾼다.
        dy = dout * self.x

        return dx, dy

### 5.2.2 덧셈 계층

In [2]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y

        return out

    def backward(self, dout): #그대로 흘려보낸다
        dx = dout * 1
        dy = dout * 1

        return dx, dy

## 5.3 활성화 함수 계층 구현하기

### 5.3.1 ReLU 
입력인 x가 0보다 크면 상류의 값을 그대로 하류로 흘려보낸다. 하지만 0 이하면 하류로 신호를 보내지 않는다(0을 보낸다).

In [3]:
class Relu:
    def __init__(self):
        self.mask = None #mask: true/false 로 구성된 넘파이

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

### 5.3.2 Sigmoid
순전파의 출력 y만으로 역전파를 계산할 수 있다.

In [4]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

## 5.4 Affine/Softmax 계층 구현하기

### 5.4.1 Affine 계층

신경망의 순전파 때 수행하는 행렬의 곱: 기하학에서는 어파인 변환 이라고 한다.  
(가중치 합 Y = np.dot(X,W)+B 처럼 계산)



### 5.4.2 배치용 Affine 계층

### 5.4.3 Softmax-with-Loss 계층

출력층에서 사용하는 소프트맥수 함수.  
소프트맥수 함수: 입력 값을 정규화하여 출력.

소프트맥스 계층을 구현할 때 손실함수인 교차 엔트로피 오차도 포함하여 softmax-with-Loss 계층이란 이름으로 구현.

In [5]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None #손실
        self.y = None #softmax의 출력
        self.t = None #정답 레이블(원-핫 코딩)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: 
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx

## 5.5 정리



1. 계산그래프는 계산 과정을 시각적으로 파악할 수 있게 해준다
2. 계산그래프의 노드는 국소적 계산. 이것들을 조합하여 전체 계산을 구성.
3. 계산그래프의 순전파는 통상의 계산 수행. 계산그래프의 역전파로는 각 노드의 미분을 구할 수 있다.
4. 신경망의 구성 요소를 계층으로 구현하여 기울기를 효율적으로 계산할 수 있다.(오차역전파법!)
5. 수치 미준과 오차역전파법의 결과를 비교하면 오차역전파법의 구현에 잘못이 없는지 확인할 수 있다. (by 기울기)